## More on OOP in Scala
1. Object type hierarchy
2. Immutable (persistent) data structures

### Object Type Hierarchy
- No primitive types in Scala - all objects inherit from Any
- Value types inherit from AnyVal, Reference types inherit from AnyRef
- Null inherits from all Reference types (null reference)
- Nothing inherits from all types (it is the "bottom" type) and cannot be instantiated

From https://www.artima.com/pins1ed/scalas-hierarchy.html


<img src="https://www.scala-lang.org/files/archive/spec/2.12/public/images/classhierarchy.png" width="800">


In [1]:
List(1, 2, 3) // what type of List?

res0: List[Int] = List(1, 2, 3)

In [2]:
List(1, 2, "three")

res1: List[Any] = List(1, 2, "three")

In [3]:
List('a', 'b')

res2: List[Char] = List('a', 'b')

In [4]:
List('a', "bee")

res3: List[Any] = List('a', "bee")

In [5]:
val x: List[AnyVal] = List(1, 2, 'a')

x: List[AnyVal] = List(1, 2, 97)

In [5]:
val x: List[AnyVal] = List(1, 2, "bee") // String does not inherit from AnyVal

cmd5.sc:1: type mismatch;
 found   : String("bee")
 required: AnyVal
Note that implicit conversions are not applicable because they are ambiguous:
 both method augmentString in object Predef of type (x: String): scala.collection.StringOps
 and method DisplayDataSyntax in object DisplayData of type (s: String): almond.interpreter.api.DisplayData.DisplayDataSyntax
 are possible conversion functions from String("bee") to AnyVal
val x: List[AnyVal] = List(1, 2, "bee") // String does not inherit from AnyVal
                                 ^Compilation Failed

: 

In [6]:
def numberOrUnit(x: Int): Any = {
    if (x < 0) {
        x // Int
    } else {
        () // Unit
    }
}

defined function numberOrUnit

In [7]:
val x = numberOrUnit(-10)
x match {
    case _: Int => println("you got a number")
    case _: Unit => println("you got a Unit")
}

you got a number


x: Any = -10

In [8]:
def returnsNever(x: Int): Nothing = {
    throw new IllegalArgumentException("No")
}

def returnsSometimes(x: Int): String = {
    if (x < 0) {
        throw new IllegalArgumentException(s"Don't give me $x")
    } else {
        x.toString
    }
}

defined function returnsNever
defined function returnsSometimes

In [9]:
returnsNever(1)

: 

In [10]:
returnsSometimes(1)

res9: String = "1"

### Immutable Data Structures
- https://en.wikipedia.org/wiki/Persistent_data_structure
- Purely Functional Data Structures by Chris Okasaki

### Exercise: Persistent Binary Search Tree

In [15]:
abstract class Tree[+A <% Ordered[A]] {
    def value: A
    def left: Tree[A]
    def right: Tree[A]
    def isLeaf: Boolean
           
    def insert[B >: A <% Ordered[B]](x: B): Tree[B] = {
        // Your code
        if (isLeaf) BSTree(x)
        else if (x < value) BSTree(value, left.insert(x), right)
        else if (x > value) BSTree(value, left, right.insert(x))
        else this
    }
        
    def contains[B >: A <% Ordered[B]](x: B): Boolean = {
        if (isLeaf) false
        else if (x < value) left.contains(x)
        else if (x > value) right.contains(x)
        else true
    }
}

case object Leaf extends Tree[Nothing] {
    def value: Nothing = throw new NotImplementedError()
    def left: Tree[Nothing] = throw new NotImplementedError()
    def right: Tree[Nothing] = throw new NotImplementedError()
    def isLeaf = true
}

case class BSTree[A <% Ordered[A]](value: A, left: Tree[A] = Leaf, right: Tree[A] = Leaf) extends Tree[A] {
    def isLeaf = false
}


defined class Tree
defined object Leaf
defined class BSTree

In [16]:
val int_tree = BSTree(4, BSTree(2))

int_tree: BSTree[Int] = BSTree(4, BSTree(2, Leaf, Leaf), Leaf)

In [17]:
int_tree.insert(3)

res16: Tree[Int] = BSTree(4, BSTree(2, Leaf, BSTree(3, Leaf, Leaf)), Leaf)

In [18]:
int_tree.insert(1)

res17: Tree[Int] = BSTree(4, BSTree(2, BSTree(1, Leaf, Leaf), Leaf), Leaf)

In [20]:
int_tree.insert(5)

res19: Tree[Int] = BSTree(4, BSTree(2, Leaf, Leaf), BSTree(5, Leaf, Leaf))

In [22]:
int_tree.contains(2)

res21: Boolean = true

In [23]:
int_tree.contains(5)

res22: Boolean = false

In [24]:
val str_tree = BSTree("aa")

str_tree: BSTree[String] = BSTree("aa", Leaf, Leaf)

In [25]:
val new_tree = str_tree.insert("bb").insert("c").insert("a")

new_tree: Tree[String] = BSTree(
  "aa",
  BSTree("a", Leaf, Leaf),
  BSTree("bb", Leaf, BSTree("c", Leaf, Leaf))
)

In [26]:
new_tree.contains("c")

res25: Boolean = true

In [27]:
str_tree.contains("c")

res26: Boolean = false